## 1. Load dataset

In [1]:
from os.path import join
import pandas as pd


def relabeler(label):
    if label == 'entailment':
        return 0
    if label == 'neutral':
        return 1
    if label == 'contradiction':
        return 2
    else:
        None


def _load(fname, genre=None, dire='data/data135628/'):
    df = pd.read_csv(join(dire, fname))
    if genre:
        genres = ['slate', 'government', 'telephone',
                  'travel', 'fiction']

        if isinstance(genre, str):
            genre = [genre]
        elif isinstance(genre, (list, tuple)):
            assert all(isinstance(g, str) for g in genre)
        else:
            raise TypeError("genre must be a str or a list of str")

        assert all(g in genres for g in genre), f'genre must be in {genres}'
        df = df[df.isin(genre)['genre']]

    df = df.fillna(" ")
    if 'test' in fname:
        df = df[['sentence1', 'sentence2']]
    else:
        df['gold_label'] = df['gold_label'].apply(relabeler)
        df.dropna(subset=['sentence1', 'sentence2', 'gold_label'], inplace=True)
        df = df[['sentence1', 'sentence2', 'gold_label']]

    out = []
    for row in df.values:
        out.append(row.tolist())
    return out


def load_dataset(fname, genre=None, dire='data/data135628/'):
    if isinstance(fname, str):
        return _load(fname, genre, dire)

    elif isinstance(fname, (list, tuple)):
        return [_load(f, genre, dire) for f in fname]

    else:
        raise TypeError("fname must be a str or a list!")

In [2]:
train_set, dev1_set, dev2_set = load_dataset(['train.csv', 'dev_matched.csv', 'dev_mismatched.csv'])
train_set = train_set + dev1_set + dev2_set

test1_set, test2_set = load_dataset(['test_matched.csv', 'test_mismatched.csv'])
print("Train set size:", len(train_set))
print("Train set examples:", train_set[:2])

print("\nTest set size:", len(test1_set))
print("nTest set examples:", test1_set[:2])

print("\nTest set size:", len(test2_set))
print("nTest set examples:", test2_set[:2])

Train set size: 412349
Train set examples: [['Conceptually cream skimming has two basic dimensions - product and geography.', 'Product and geography are what make cream skimming work. ', 1], ['you know during the season and i guess at at your level uh you lose them to the next level if if they decide to recall the the parent team the Braves decide to call to recall a guy from triple A then a double A guy goes up to replace him and a single A guy goes up to replace him', 'You lose the things to the following level if the people recall.', 0]]

Test set size: 9796
nTest set examples: [['That which binds together Chinese.', 'This is a shared value among Chinese people.'], ["The actual length of an individual worker's H-2A visa varies depending upon the geographic location of the employer and the nature of the farmwork to be performed.", "The location of the employer effects the length of the worker's H-2A visa."]]

Test set size: 9847
nTest set examples: [['Even here, the channel perspecti

## 2. Transform text

In [3]:
from paddlenlp.datasets import MapDataset
from paddle.io import BatchSampler, DataLoader
from paddlenlp.data import Pad, Stack, Tuple
from paddlenlp.transformers import BertModel as SeqClfModel
from paddlenlp.transformers import BertTokenizer as PTMTokenizer


MODEL_NAME = "bert-base-uncased"
tokenizer = PTMTokenizer.from_pretrained(MODEL_NAME)


def example_converter(example, max_seq_length, tokenizer):
    text_a, text_b, label = example
    encoded = tokenizer(text=text_a, text_pair=text_b, max_seq_len=max_seq_length)
    input_ids = encoded["input_ids"]
    token_type_ids = encoded["token_type_ids"]
    return input_ids, token_type_ids, label


def get_trans_fn(max_seq_length=128, tokenizer=tokenizer):
    return lambda ex: example_converter(ex, max_seq_length, tokenizer)


batchify_fn = lambda samples, fn=Tuple(
    Pad(axis=0, pad_val=tokenizer.pad_token_id),
    Pad(axis=0, pad_val=tokenizer.pad_token_type_id),
    Stack(dtype="int64")
    ): fn(samples)


def create_dataloader(dataset,
                      trans_fn,
                      batchify_fn,
                      test=False,
                      batch_size=128,
                      shuffle=True,
                      sampler=BatchSampler):

    if test:
        dataset = [d + [0] for d in dataset]

    if not isinstance(dataset, MapDataset):
        dataset = MapDataset(dataset)

    dataset.map(trans_fn)
    batch_sampler = sampler(dataset,
                            shuffle=shuffle,
                            batch_size=batch_size)

    dataloder = DataLoader(dataset,
                           batch_sampler=batch_sampler,
                           collate_fn=batchify_fn)

    return dataloder

[2022-03-30 01:21:26,811] [    INFO]

 - Downloading https://paddle-hapi.bj.bcebos.com/models/bert/bert-base-uncased-vocab.txt and saved to /home/aistudio/.paddlenlp/models/bert-base-uncased

[2022-03-30 01:21:26,814] [    INFO]

 - Downloading bert-base-uncased-vocab.txt from https://paddle-hapi.bj.bcebos.com/models/bert/bert-base-uncased-vocab.txt

  0%|          | 0/227 [00:00<?, ?it/s]

100%|██████████| 227/227 [00:00<00:00, 23654.24it/s]

In [4]:
max_seq_length = 128; batch_size = 64
trans_fn = get_trans_fn(max_seq_length)
train_loader = create_dataloader(train_set, trans_fn, batchify_fn, batch_size=batch_size)
# dev_loader = create_dataloader(dev_set, trans_fn, batchify_fn, batch_size=batch_size)
test1_loader = create_dataloader(test1_set, trans_fn, batchify_fn, test=True, shuffle=False, batch_size=batch_size)
test2_loader = create_dataloader(test2_set, trans_fn, batchify_fn, test=True, shuffle=False, batch_size=batch_size)

## 3. Model building

In [5]:
from paddle import nn
import paddle


class PTM(nn.Layer):

    def __init__(self, pretrained_model, dropout=0.1, num_class=3):
        super().__init__()

        self.ptm = pretrained_model
        ptm_out_dim = self.ptm.config["hidden_size"]
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(ptm_out_dim, ptm_out_dim // 2)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(ptm_out_dim // 2, num_class)

    def encoder(self, input_ids, token_type_ids):
        _, embd = self.ptm(input_ids, token_type_ids)
        embd = self.dropout(embd)
        return embd

    def forward(self, input_ids, token_type_ids):
        embd = self.encoder(input_ids, token_type_ids)
        hidden = self.relu(self.fc1(embd))
        logits = self.fc2(hidden)
        return logits

In [6]:
from paddlenlp.transformers import LinearDecayWithWarmup

epoch = 3
weight_decay = 0.0
warmup_proportion = 0.0
lr_scheduler = LinearDecayWithWarmup(5e-5, len(train_loader) * epoch,
                                         warmup_proportion)

def get_model(model):
    decay_params = [
        p.name for n, p in model.named_parameters()
        if not any(nd in n for nd in ["bias", "norm"])
    ]
    optimizer = paddle.optimizer.AdamW(
    parameters=model.parameters(),
    learning_rate=lr_scheduler,
    weight_decay=weight_decay,
    apply_decay_param_fun=lambda x: x in decay_params)

    criterion = paddle.nn.CrossEntropyLoss()

    model = paddle.Model(model)
    metric = paddle.metric.Accuracy()
    model.prepare(optimizer, criterion, metric)
    return model

In [7]:
model = SeqClfModel.from_pretrained(MODEL_NAME)
model = PTM(model)
model = get_model(model)

[2022-03-30 01:21:27,018] [    INFO]

 - Downloading https://paddlenlp.bj.bcebos.com/models/transformers/bert-base-uncased.pdparams and saved to /home/aistudio/.paddlenlp/models/bert-base-uncased

[2022-03-30 01:21:27,020] [    INFO]

 - Downloading bert-base-uncased.pdparams from https://paddlenlp.bj.bcebos.com/models/transformers/bert-base-uncased.pdparams

  0%|          | 0/793257 [00:00<?, ?it/s]

...

100%|█████████▉| 790263/793257 [00:11<00:00, 73714.47it/s]

100%|██████████| 793257/793257 [00:11<00:00, 67497.35it/s]

W0330 01:21:38.848248   255 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W0330 01:21:38.853195   255 device_context.cc:465] device: 0, cuDNN Version: 7.6.


[2022-03-30 01:21:49,124] [    INFO]

 - Weights from pretrained model not used in BertModel: ['cls.predictions.decoder_weight', 'cls.predictions.decoder_bias', 'cls.predictions.transform.weight', 'cls.predictions.transform.bias', 'cls.predictions.layer_norm.weight', 'cls.predictions.layer_norm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']

## 4. Model training

In [8]:
model.fit(train_loader, epochs=epoch, verbose=2, log_freq=100)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.

Epoch 1/3

step  100/6443 - loss: 0.7668 - acc: 0.5563 - 361ms/step


step  200/6443 - loss: 0.5352 - acc: 0.6214 - 359ms/step


step  300/6443 - loss: 0.8902 - acc: 0.6503 - 355ms/step


...

step 6100/6443 - loss: 0.0658 - acc: 0.9376 - 357ms/step


step 6200/6443 - loss: 0.1126 - acc: 0.9376 - 357ms/step


step 6300/6443 - loss: 0.1005 - acc: 0.9375 - 357ms/step


step 6400/6443 - loss: 0.1572 - acc: 0.9375 - 358ms/step


step 6443/6443 - loss: 0.1478 - acc: 0.9375 - 357ms/step


## 5. Prediction

In [9]:
import paddle.nn.functional as F
from tqdm import tqdm


def predict(test_loader, id_inpath, out):
    predictions = []
    logits = model.predict(test_loader)

    for batch in tqdm(logits[0]):
        batch = paddle.to_tensor(batch)
        probs = F.softmax(batch, axis=1)
        preds = paddle.argmax(probs, axis=1).numpy().tolist()
        predictions.extend(preds)

    df = pd.read_csv(id_inpath)
    ids = df['pairID'].tolist()
    label_map = {0: 'entailment', 1: 'neutral', 2: 'contradiction'}
    results = [[idx, label_map[p]] for idx, p in zip(ids, predictions)]
    columns = ['pairID', 'glod_label']
    pd.DataFrame(results, columns=columns).to_csv(out, index=False)
    print(f"{out} has been saved!")

In [10]:
predict(test1_loader, 'data/data135628/test_matched.csv', 'test_matched_preds.csv')

Predict begin...

step   2/154 [..............................]

 - ETA: 35s - 233ms/step



step   4/154 [..............................]

...

 - 148ms/step          


Predict samples: 9796

  0%|          | 0/154 [00:00<?, ?it/s]

100%|██████████| 154/154 [00:00<00:00, 8663.48it/s]

test_matched_preds.csv has been saved!

In [11]:
predict(test2_loader, 'data/data135628/test_mismatched.csv', 'test_mismatched_preds.csv')

Predict begin...

step   2/154 [..............................]

 - ETA: 34s - 230ms/step



step   4/154 [..............................]

...

 - 148ms/step          


Predict samples: 9847

  0%|          | 0/154 [00:00<?, ?it/s]

100%|██████████| 154/154 [00:00<00:00, 8578.33it/s]

test_mismatched_preds.csv has been saved!